In [22]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers

In [32]:
# tf.keras.utils.get_file('petfinder_mini.zip', dataset_url,
#                        extract=True, cache_dir='.')
csv_file = 'eda_ds.csv'
df = pd.read_csv(csv_file)
df.head()

,tu_fico_9,tradeline_count,inquiry_count,collection_count,revolving_utilization,delinquent_secured,delinquent_unsecured,bankruptcy_count,stated_indiv_income,stated_additional_income,tu_income_estimate,tu_monthly_payments,tu_mortgage_payments,riskview,principal,cpb,days_delinquent,payment_history
0,498,1,0,1,0,0,0,0,24000,0,35,46,0,NaN,NaN,NaN,NaN,open;2019-05-10;2021-04-03;1543;0;0;1543;indiv...
1,612,7,5,1,63,0,0,0,30000,0,29,126,0,706.0,NaN,NaN,NaN,installment;2019-10-05;2019-11-03;0;352;0;0;in...
2,617,21,8,0,32,0,0,0,60000,0,39,1186,1075,779.0,NaN,NaN,NaN,installment;2013-06-11;2016-10-11;0;5000;0;0;i...
3,647,10,1,0,64,0,0,0,400000,0,68,649,0,724.0,NaN,NaN,NaN,installment;2009-09-18;2013-06-24;0;35599;0;0;...
4,647,16,7,4,56,0,0,0,65000,0,55,1074,0,660.0,NaN,NaN,NaN,installment;2007-10-19;2011-09-26;0;14162;0;0;...


In [37]:
df2         = df.copy()
col         = 'revolving_utilization'
conditions  = [ df2[col] >= 80, (df2[col] < 80) & (df2[col]>= 60), (df2[col] < 60) & (df2[col]>= 40), (df2[col] < 40) & (df2[col]>= 20), df2[col] < 20 ]
choices     = [ "very-high", 'high', 'medium', 'low', 'very-low' ]
    
df2["revolving_utilization_cat"] = np.select(conditions, choices, default=np.nan)

In [38]:
df2.head()

,tu_fico_9,tradeline_count,inquiry_count,collection_count,revolving_utilization,delinquent_secured,delinquent_unsecured,bankruptcy_count,stated_indiv_income,stated_additional_income,tu_income_estimate,tu_monthly_payments,tu_mortgage_payments,riskview,principal,cpb,days_delinquent,payment_history,revolving_utilization_cat
0,498,1,0,1,0,0,0,0,24000,0,35,46,0,NaN,NaN,NaN,NaN,open;2019-05-10;2021-04-03;1543;0;0;1543;indiv...,very-low
1,612,7,5,1,63,0,0,0,30000,0,29,126,0,706.0,NaN,NaN,NaN,installment;2019-10-05;2019-11-03;0;352;0;0;in...,high
2,617,21,8,0,32,0,0,0,60000,0,39,1186,1075,779.0,NaN,NaN,NaN,installment;2013-06-11;2016-10-11;0;5000;0;0;i...,low
3,647,10,1,0,64,0,0,0,400000,0,68,649,0,724.0,NaN,NaN,NaN,installment;2009-09-18;2013-06-24;0;35599;0;0;...,high
4,647,16,7,4,56,0,0,0,65000,0,55,1074,0,660.0,NaN,NaN,NaN,installment;2007-10-19;2011-09-26;0;14162;0;0;...,medium


In [ ]:
# New column not based on existing
# df.with_columns(new_column = pl.lit('some_text'))

In [40]:
# In the original dataset, `'AdoptionSpeed'` of `4` indicates
# a pet was not adopted.
df2['multiple_collections'] = np.where(df['collection_count']>=1, 0, 1)

# Drop unused features.
df = df2.drop(columns=['delinquent_unsecured'])

## Split the DataFrame into training, validation, and test sets

In [41]:
train, val, test = np.split(df.sample(frac=1), [int(0.8*len(df)), int(0.9*len(df))])

In [42]:
print(len(train), 'training examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

79 training examples
10 validation examples
10 test examples


## Create input pipeline using tf.data
Create a utility function that converts each training, validation, and test set DataFrame into a `tf.data.Dataset`, then shuffles and batches the data.
If you were working with a very large CSV file (so large that it does not fit into memory), you'd use the `tf.data` API to read it from disk directly.

In [45]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  df = dataframe.copy()
  labels = df.pop('multiple_collections')
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

Now, use newly created function (`df_to_dataset`) to check format of data that input pipeline helper function returns by calling it on the training data, & use a small batch size to keep the output readable:

In [46]:
batch_size = 5
train_ds = df_to_dataset(train, batch_size=batch_size)

/tmp/ipykernel_23099/2467672315.py:4: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in dataframe.items()}


In [47]:
[(train_features, label_batch)] = train_ds.take(1)
print('Every feature:', list(train_features.keys()))
print('A batch of tradeline_count:', train_features['tradeline_count'])
print('A batch of targets:', label_batch )

Every feature: ['tu_fico_9', 'tradeline_count', 'inquiry_count', 'collection_count', 'revolving_utilization', 'delinquent_secured', 'bankruptcy_count', 'stated_indiv_income', 'stated_additional_income', 'tu_income_estimate', 'tu_monthly_payments', 'tu_mortgage_payments', 'riskview', 'principal', 'cpb', 'days_delinquent', 'payment_history', 'revolving_utilization_cat', 'multiple_collections']
A batch of tradeline_count: tf.Tensor(
[[ 9]
 [18]
 [ 9]
 [14]
 [31]], shape=(5, 1), dtype=int64)
A batch of targets: tf.Tensor([1 1 0 0 0], shape=(5,), dtype=int64)


2023-05-10 01:42:22.146218: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_5' with dtype int64 and shape [79,1]
	 [[{{node Placeholder/_5}}]]
2023-05-10 01:42:22.149669: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [79,1]
	 [[{{node Placeholder/_4}}]]


In [48]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [49]:
tradeline_count_col = train_features['tradeline_count']
layer = get_normalization_layer('tradeline_count', train_ds)
layer(tradeline_count_col)

2023-05-10 01:42:30.881562: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype int64 and shape [79,1]
	 [[{{node Placeholder/_1}}]]
2023-05-10 01:42:30.883353: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_16' with dtype int64 and shape [79,1]
	 [[{{node Placeholder/_16}}]]


<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.49590552],
       [ 0.13597389],
       [-0.49590552],
       [-0.1448614 ],
       [ 1.0486885 ]], dtype=float32)>

Note: If you have many numeric features (hundreds, or more), it is more efficient to concatenate them first and use a single `tf.keras.layers.Normalization` layer.

In [50]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

In [51]:
test_type_col = train_features['revolving_utilization_cat']
test_type_layer = get_category_encoding_layer(name='revolving_utilization_cat',
                                              dataset=train_ds,
                                              dtype='string')
test_type_layer(test_type_col)

2023-05-10 01:43:23.535089: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_7' with dtype string and shape [79,1]
	 [[{{node Placeholder/_7}}]]
2023-05-10 01:43:23.537108: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype int64 and shape [79,1]
	 [[{{node Placeholder/_17}}]]


<tf.Tensor: shape=(5, 6), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0., 0.]], dtype=float32)>

Earlier, you used a small batch size to demonstrate the input pipeline. Let's now create a new input pipeline with a larger batch size of 256:

In [ ]:
batch_size = 256
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)